## Import needed Libs

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

## Laden und mit den Daten spielen

In [4]:
df = pd.read_csv('../Daten/autoscout24-germany-dataset.csv')

print(df.shape)

df.head(7)

(46405, 9)


,mileage,make,model,fuel,gear,offerType,price,hp,year
0,235000,BMW,316,Diesel,Manual,Used,6800,116.0,2011
1,92800,Volkswagen,Golf,Gasoline,Manual,Used,6877,122.0,2011
2,149300,SEAT,Exeo,Gasoline,Manual,Used,6900,160.0,2011
3,96200,Renault,Megane,Gasoline,Manual,Used,6950,110.0,2011
4,156000,Peugeot,308,Gasoline,Manual,Used,6950,156.0,2011
5,147000,Toyota,Auris,Electric/Gasoline,Automatic,Used,6950,99.0,2011
6,91894,Renault,Scenic,Diesel,Manual,Used,6970,131.0,2011


## Zählen na und doppelten Daten (um sie später zu entfernen)

In [5]:
print(f"There is: {df.isnull().sum().sum()} missing values in the dataset")

print(f"There is: {df.duplicated().sum()} duplicates in the dataset")

There is: 354 missing values in the dataset
There is: 2140 duplicates in the dataset


# Datenbereinigung

In [6]:
# Entfernen von doppelten und fehlenden Daten

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

## Daten nach Bereinigung

In [7]:
print(df.shape)

df.info

(43947, 9)


<bound method DataFrame.info of        mileage        make   model               fuel    gear       offerType  \
0       235000         BMW     316             Diesel  Manual            Used   
1        92800  Volkswagen    Golf           Gasoline  Manual            Used   
2       149300        SEAT    Exeo           Gasoline  Manual            Used   
3        96200     Renault  Megane           Gasoline  Manual            Used   
4       156000     Peugeot     308           Gasoline  Manual            Used   
...        ...         ...     ...                ...     ...             ...   
46394       10     Citroen      C1           Gasoline  Manual  Pre-registered   
46396       99        Fiat     500  Electric/Gasoline  Manual  Pre-registered   
46397      550        Fiat     500  Electric/Gasoline  Manual   Demonstration   
46398      837        Fiat   Panda  Electric/Gasoline  Manual   Demonstration   
46399     1500       Skoda   Fabia           Gasoline  Manual   Demonstration

## Die Fahrzeugtypen (Gear)

In [8]:
gear = df['gear'].value_counts();
gear.head()

gear
Manual            28752
Automatic         15141
Semi-automatic       54
Name: count, dtype: int64

## Teuersten Autos auf dem deutschen Automarkt

In [9]:
teuer_autos = df[['make', 'model', 'price']].sort_values('price', ascending=False).head(10)
teuer_autos

,make,model,price
21675,Ferrari,F12,1199900
11754,Mercedes-Benz,S 650,717078
11753,Maybach,Pullman,717078
16429,Maybach,Pullman,499800
29257,Mercedes-Benz,SLS,465000
7384,Lamborghini,Aventador,449900
18419,Ferrari,812,439900
7383,Lamborghini,Aventador,439900
2170,Lamborghini,Aventador,399999
42132,Porsche,991,399911


## Gesamtzahl der Fahrzeuge je Hersteller

In [10]:
df['make'].value_counts()

make
Volkswagen    6680
Opel          4576
Ford          4245
Skoda         2737
Renault       2659
              ... 
Isuzu            1
Others           1
Zhidou           1
Brilliance       1
Alpine           1
Name: count, Length: 71, dtype: int64

## Wirkt sich der Preis auf die Nutzung des Autos aus?

* Die Korrelation ist negativ und beträgt etwa -0.31.

* Das heißt: Je höher die Laufleistung, desto niedriger ist tendenziell der Preis (und umgekehrt).

* Der Zusammenhang ist moderat (nicht sehr stark, aber auch nicht vernachlässigbar).

In [11]:
print("\nBasis-Korrelationsanalyse:")

korr_Ma = df[['price', 'mileage']].corr()
print(korr_Ma)


Basis-Korrelationsanalyse:
            price   mileage
price    1.000000 -0.305598
mileage -0.305598  1.000000


##  bestes Jahr für den Autoverkauf

In [12]:
jahr_za = df.groupby('year')['make'].count()

sort = jahr_za.sort_values(ascending=False)
print(sort)


year
2013    4096
2016    4087
2020    4087
2018    4078
2017    4072
2012    4063
2015    4054
2019    4031
2014    4021
2011    3984
2021    3374
Name: make, dtype: int64


## Beliebtheit der Kraftstoffart im Laufe der Zeit

In [13]:
print("Häufigste Kraftstoffart pro Jahr:")
fuel_year = df.groupby(['year', 'fuel']).size().reset_index(name='count')
beliebt_fuel = fuel_year.loc[fuel_year.groupby('year')['count'].idxmax()]
print(beliebt_fuel)

Häufigste Kraftstoffart pro Jahr:
    year      fuel  count
6   2011  Gasoline   2564
15  2012  Gasoline   2530
24  2013  Gasoline   2541
33  2014  Gasoline   2471
42  2015  Gasoline   2424
51  2016  Gasoline   2422
60  2017  Gasoline   2264
70  2018  Gasoline   2341
78  2019  Gasoline   2653
87  2020  Gasoline   2613
96  2021  Gasoline   2423


## Wie entwickeln sich Preise über die Jahre?

In [14]:
durch_price = df.groupby('year')['price'].mean().sort_index()
print(durch_price)

year
2011     7063.175954
2012     8226.920502
2013     9264.150635
2014     9986.855011
2015    11170.919832
2016    12749.987521
2017    18394.968075
2018    22123.274890
2019    24823.363930
2020    29165.813800
2021    31333.808239
Name: price, dtype: float64


## Kosten Automatikautos mehr als Schaltgetriebe?

In [15]:
auto_prices = df[df['gear'] == 'Automatic']['price']
manual_prices = df[df['gear'] == 'Manual']['price']

t_stat_auto, p_val_auto = ttest_ind(auto_prices, manual_prices, equal_var=False, nan_policy='omit')

print(f"Durchschnittspreis Automatik: €{auto_prices.mean():,.2f}")
print(f"Durchschnittspreis Schaltgetriebe: €{manual_prices.mean():,.2f}")
print(f"T-Statistik: {t_stat_auto:.2f}")

Durchschnittspreis Automatik: €28,054.26
Durchschnittspreis Schaltgetriebe: €10,501.88
T-Statistik: 74.82


### Korrelationsmatrix für alle numerischen Spalten

In [25]:
# 1. Nur numerische Spalten auswählen
numerische_df = df.select_dtypes(include=np.number)  # Oder ['price', 'mileage', 'year', ...] für bestimmte Spalten
print("Numerische Spalten für Korrelation:", numerische_df.columns)  # Überprüfe die ausgewählten Spalten

korrelationsmatrix = numerische_df.corr()
print("\nKorrelationsmatrix:")
print(korrelationsmatrix)


Numerische Spalten für Korrelation: Index(['mileage', 'price', 'hp', 'year', 'age', 'price_per_mileage',
       'cluster'],
      dtype='object')

Korrelationsmatrix:
                    mileage     price        hp      year       age  \
mileage            1.000000 -0.305598 -0.022912 -0.670614  0.670614   
price             -0.305598  1.000000  0.745003  0.411415 -0.411415   
hp                -0.022912  0.745003  1.000000  0.181946 -0.181946   
year              -0.670614  0.411415  0.181946  1.000000 -1.000000   
age                0.670614 -0.411415 -0.181946 -1.000000  1.000000   
price_per_mileage -0.150113  0.124175  0.050312  0.196366 -0.196366   
cluster            0.638856  0.115566  0.256239 -0.440677  0.440677   

                   price_per_mileage   cluster  
mileage                    -0.150113  0.638856  
price                       0.124175  0.115566  
hp                          0.050312  0.256239  
year                        0.196366 -0.440677  
age                

### Pearson-Korrelation

In [ ]:
print("\n2. Pearson-Korrelation")
korrelation_pearson = df['price'].corr(df['mileage'], method='pearson')
print(f"   Pearson-Korrelation zwischen Preis und Kilometerstand: {korrelation_pearson:.2f}")

### Spearman-Korrelation

In [ ]:
print("\n3. Spearman-Korrelation")
korrelation_spearman = df['price'].corr(df['mileage'], method='spearman')
print(f"   Spearman-Korrelation zwischen Preis und Kilometerstand: {korrelation_spearman:.2f}")

### Kendall-Korrelation

In [ ]:
print("\n4. Kendall-Korrelation")
korrelation_kendall = df['price'].corr(df['mileage'], method='kendall')
print(f"   Kendall-Korrelation zwischen Preis und Kilometerstand: {korrelation_kendall:.2f}")

Ausreißer im Preis

In [24]:
from scipy.stats import ttest_ind, f_oneway
from datetime import datetime

# 1. Datenbereinigung (wie in deinem Notebook)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

# 2. Erweiterte Datenanalyse und Feature Engineering
def finde_ausreisser_iqr(data, spalte):
    Q1 = data[spalte].quantile(0.25)
    Q3 = data[spalte].quantile(0.75)
    IQR = Q3 - Q1
    untergrenze = Q1 - 1.5 * IQR
    obergrenze = Q3 + 1.5 * IQR
    ausreisser = data[(data[spalte] < untergrenze) | (data[spalte] > obergrenze)]
    return ausreisser

ausreisser_preis = finde_ausreisser_iqr(df, 'price')
print("\nAusreißer im Preis:")
print(ausreisser_preis.head())

aktuelles_jahr = datetime.now().year
df['age'] = aktuelles_jahr - df['year']
df['price_per_mileage'] = df['price'] / df['mileage']





Ausreißer im Preis:
     mileage           make       model      fuel       gear offerType  price  \
120    32100  Mercedes-Benz       E 400  Gasoline  Automatic      Used  37931   
143    85200     Volkswagen      Amarok    Diesel  Automatic      Used  37700   
144    49900           Audi  A6 allroad    Diesel  Automatic      Used  37950   
145    39580           Audi          Q5    Diesel  Automatic      Used  39444   
146    49394           Audi          Q5    Diesel  Automatic      Used  42480   

        hp  year  age  price_per_mileage  cluster  
120  333.0  2017    8           1.181651        0  
143  224.0  2018    7           0.442488        0  
144  320.0  2018    7           0.760521        0  
145  190.0  2018    7           0.996564        0  
146  190.0  2018    7           0.860023        0  


### ANOVA - Preisunterschiede nach Getriebe

In [23]:
# 5. Statistische Tests
gruppen = df['gear'].unique()
gruppen_preise = [df[df['gear'] == gruppe]['price'] for gruppe in gruppen]

f_wert, p_wert = f_oneway(*gruppen_preise)
print("\nANOVA - Preisunterschiede nach Getriebe:")
print(f"F-Wert: {f_wert:.2f}, p-Wert: {p_wert:.3f}")

# 6. Ausgabe der ersten Zeilen des bearbeiteten DataFrames
print("\nErste Zeilen des bearbeiteten DataFrames:")
print(df.head())


ANOVA - Preisunterschiede nach Getriebe:
F-Wert: 4956.96, p-Wert: 0.000

Erste Zeilen des bearbeiteten DataFrames:
   mileage        make   model      fuel    gear offerType  price     hp  \
0   235000         BMW     316    Diesel  Manual      Used   6800  116.0   
1    92800  Volkswagen    Golf  Gasoline  Manual      Used   6877  122.0   
2   149300        SEAT    Exeo  Gasoline  Manual      Used   6900  160.0   
3    96200     Renault  Megane  Gasoline  Manual      Used   6950  110.0   
4   156000     Peugeot     308  Gasoline  Manual      Used   6950  156.0   

   year  age  price_per_mileage  cluster  
0  2011   14           0.028936        1  
1  2011   14           0.074106        1  
2  2011   14           0.046216        1  
3  2011   14           0.072245        1  
4  2011   14           0.044551        1  
